In [7]:
#필요한 모듈 임포트하기
import pandas as pd
from pandas import DataFrame  as df
import numpy as np
from collections import Counter
from konlpy.tag import Okt
okt = Okt()
%matplotlib inline
import matplotlib.pyplot as plt
import re 
import gensim
from gensim import corpora, models
from gensim.models import CoherenceModel
from wordcloud import WordCloud
from datetime import datetime


In [6]:
!pip install konlpy

     |████████████████████████████████| 19.4MB 1.2MB/s 
     |████████████████████████████████| 92kB 11.6MB/s 
     |████████████████████████████████| 460kB 51.5MB/s 
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [25]:
# test dataset upload
from google.colab import files
myfile = files.upload()

Saving friends1.csv to friends1.csv


In [26]:
Data= pd.read_csv('ko_data.csv',encoding ='cp949')

In [27]:
Data

,Id,Sentence
0,0,정말 많이 울었던 영화입니다.
1,1,시간 낭비예요.
2,2,포스터를 저렇게밖에 만들지 못했던 제작자의 소심함에 침을 뱉고 싶다.
3,3,지금 봐도 재미있는 영화!!! 코믹과 감동!!! 그리고 요리!!!
4,4,이걸 영화로 만드는 거야?얼마나 가는지 보자.
...,...,...
11182,11182,이 영화를 커플에게 추천합니다. 영화관에 가다보면 평생 잊지 못할 추억이 하나 생길...
11183,11183,심심__ 그냥 한효주 cf
11184,11184,공감해서 눈물나는 영화. 안 보신분들이 전부 제가 울었다고 하면 의아해하실텐데 보면...
11185,11185,오토바이 신은 최고네요.


In [30]:
#데이터 프레임의 'Sentence' 열의 값들을 str 형식으로 바꾸기
Data.Sentence = Data.Sentence.astype(str)


#데이터 프레임의 'Sentence' 열의 값 중 keyword1이나 keyword 2가 포함된 행만 Data에 저장
#clean_Data = Data.loc[Data['Sentence'].str.contains('keyword1|keyword2')]

#데이터 프레임의 'Sentence' 열의 값 중 keyword1이나 keyword 2가 포함된 행은 삭제
#clean_Data = Data[~Data['Sentence'].str.contains('keyword1|keyword2')]


#Sentence와 열을 기준으로 중복된 데이터를 삭제, inplace : 데이터 프레임을 변경할지 선택(원본을 대체)
clean_Data.drop_duplicates(subset=['Sentence'], inplace=True)

#한글이 아니면 빈 문자열로 바꾸기
clean_Data['Sentence'] = clean_Data['Sentence'].str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣]',' ',regex=True)

#빈 문자열 NAN 값으로 바꾸기
clean_Data = clean_Data.replace({'': np.nan})
clean_Data = clean_Data.replace(r'^\s*$', None, regex=True)

#NAN 이 있는 행은 삭제
clean_Data.dropna(how='any', inplace=True)

#인덱스 차곡차곡
clean_Data = clean_Data.reset_index (drop = True)

#데이터 프레임에 null 값이 있는지 확인
print(clean_Data.isnull().values.any()) 

False


In [31]:
#텍스트 데이터를 리스트로 변환
Data_list=clean_Data.Sentence.values.tolist()

#리스트를 요소별로 가져와서 명사만 추출한 후 리스트로 저장
data_word=[]
for i in range(len(Data_list)):
    try:
        data_word.append(okt.nouns(Data_list[i]))
    except Exception as e:
        continue

#명사만 추출해서 만든 리스트
data_word

[['정말', '영화'],
 ['시간', '낭비'],
 ['포스터', '제작자', '침'],
 ['지금', '영화', '코믹', '감동', '요리'],
 ['걸', '영화로', '얼마나'],
 ['감동', '영화', '영화', '줄리', '안', '무어', '매력'],
 ['프랑스', '영화', '정말', '말'],
 ['이보', '더', '수'],
 ['잠', '자고'],
 ['오프닝', '씬', '이', '영화', '젊음', '대해', '말'],
 ['내용', '유덕화', '가장', '영화'],
 ['영화', '색깔'],
 ['홍성', '진', '영화', '해설', '님', '우뢰매', '욕'],
 ['이'],
 ['작가', '주의', '척'],
 [],
 ['최고', '녹화', '걸', '볼', '수', '정말', '다행'],
 ['동감', '해', '남', '것', '수고'],
 ['역시', '강', '씨'],
 ['정말'],
 ['진', '얼굴', '몸짱', '정말'],
 ['감동', '감성', '자극'],
 ['힘', '영화'],
 ['우리', '은별', '언니'],
 ['분위기'],
 ['소설', '듯', '줄거리', '볼썽', '액션'],
 ['충훈이'],
 ['보고', '좀'],
 [],
 ['말', '간접', '광고', '보양', '생'],
 ['예술', '영화', '스토리', '스케일'],
 ['언어', '최고', '애니메이션'],
 ['사랑', '앞', '돈', '못', '보신', '분', '꼭', '추천'],
 ['보시', '별루임', '알', '수'],
 ['최고', '영화'],
 ['보지'],
 ['이', '영화', '지루함', '우리', '말로', '표현', '수', '뭔가', '것', '결코'],
 ['접', '추천', '아주', '해프닝', '사건'],
 ['전혀'],
 ['점도'],
 ['오히려', '관객', '돈', '영화'],
 ['쓰레기', '영화', '극장판', '돈', '얼마나', '알', '수'],
 ['정말', '

In [54]:
from gensim import corpora
dictionary = corpora.Dictionary(data_word)
corpus = [dictionary.doc2bow(text) for text in data_word]
print(corpus[1]) # 수행된 결과에서 두번째 뉴스 출력. 첫번째 문서의 인덱스는 0

[(2, 1), (3, 1)]


In [55]:
print(dictionary[66])

소설


In [56]:
len(dictionary)

8299

In [58]:
import gensim
NUM_TOPICS = 20 #20개의 토픽, k=20
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

(0, '0.226*"정말" + 0.089*"영화" + 0.067*"나" + 0.043*"쓰레기"')
(1, '0.158*"최고" + 0.059*"작품" + 0.058*"영화" + 0.057*"중"')
(2, '0.076*"사람" + 0.029*"전개" + 0.026*"매력" + 0.023*"속"')
(3, '0.062*"이야기" + 0.038*"걸" + 0.014*"머리" + 0.014*"제발"')
(4, '0.128*"수" + 0.048*"영화" + 0.039*"것" + 0.028*"여자"')
(5, '0.070*"좀" + 0.066*"거" + 0.065*"뭐" + 0.041*"끝"')
(6, '0.103*"때" + 0.051*"영화" + 0.040*"볼" + 0.022*"이상"')
(7, '0.088*"보고" + 0.049*"또" + 0.033*"소재" + 0.026*"원작"')
(8, '0.046*"반전" + 0.042*"웃음" + 0.034*"모두" + 0.021*"때문"')
(9, '0.091*"시간" + 0.083*"영화" + 0.066*"하나" + 0.023*"말"')
(10, '0.049*"정도" + 0.045*"다시" + 0.044*"명작" + 0.044*"년"')
(11, '0.081*"영화" + 0.046*"돈" + 0.037*"번" + 0.026*"눈"')
(12, '0.203*"영화" + 0.150*"이" + 0.022*"지금" + 0.021*"완전"')
(13, '0.104*"생각" + 0.071*"영화" + 0.041*"마음" + 0.034*"추천"')
(14, '0.079*"내용" + 0.056*"별로" + 0.048*"아이" + 0.042*"감독"')
(15, '0.075*"그냥" + 0.046*"게" + 0.040*"연출" + 0.033*"연기"')
(16, '0.151*"점" + 0.120*"평점" + 0.077*"왜" + 0.030*"느낌"')
(17, '0.110*"안" + 0.096*"말" + 0.046*"줄" + 0.

In [59]:
print(ldamodel.print_topics())

[(0, '0.226*"정말" + 0.089*"영화" + 0.067*"나" + 0.043*"쓰레기" + 0.033*"최악" + 0.026*"결말" + 0.017*"여운" + 0.014*"굿" + 0.013*"연기" + 0.012*"음"'), (1, '0.158*"최고" + 0.059*"작품" + 0.058*"영화" + 0.057*"중" + 0.021*"가장" + 0.016*"내" + 0.015*"라면" + 0.015*"가족" + 0.011*"연기" + 0.011*"위"'), (2, '0.076*"사람" + 0.029*"전개" + 0.026*"매력" + 0.023*"속" + 0.022*"영화" + 0.020*"애니메이션" + 0.019*"제목" + 0.018*"보지" + 0.018*"개봉" + 0.017*"로맨스"'), (3, '0.062*"이야기" + 0.038*"걸" + 0.014*"머리" + 0.014*"제발" + 0.012*"대사" + 0.012*"한국영" + 0.012*"곳" + 0.011*"바보" + 0.011*"해" + 0.011*"예고편"'), (4, '0.128*"수" + 0.048*"영화" + 0.039*"것" + 0.028*"여자" + 0.024*"알" + 0.023*"액션" + 0.023*"주인공" + 0.022*"볼" + 0.021*"현실" + 0.021*"인간"'), (5, '0.070*"좀" + 0.066*"거" + 0.065*"뭐" + 0.041*"끝" + 0.036*"이건" + 0.029*"영화" + 0.018*"점수" + 0.017*"것" + 0.015*"평론가" + 0.014*"실망"'), (6, '0.103*"때" + 0.051*"영화" + 0.040*"볼" + 0.022*"이상" + 0.019*"공감" + 0.018*"것" + 0.018*"긴장감" + 0.017*"급" + 0.017*"매우" + 0.016*"역시"'), (7, '0.088*"보고" + 0.049*"또" + 0.033*"소재" + 0.026*"원작" + 0.0

In [60]:
pip install pyLDAvis

     |████████████████████████████████| 1.6MB 12.2MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97712 sha256=289427880ec6cbcde976480d1aa3e46023eb58f442d3468fc95ffcd149e18c83
  Stored in directory: /root/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
Successfully built pyLDAvis


In [61]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)
pyLDAvis.display(vis)